In [ ]:
import sys

sys.path.append("..")

In [ ]:
from src.memory import MemoryLayer

In [ ]:
# Create with branching factor of 3 for quick testing
ml = MemoryLayer(branching_factor=3)

# Add 10 entries - should trigger multiple rollups
for i in range(10):
    ml.add(f"Entry {i}")

print(ml.stats())

## Test 2: View top nodes (most summarized)

In [ ]:
top_nodes = ml.top()
print(f"Top-level nodes: {len(top_nodes)}")

for node_id in top_nodes:
    level = ml.graph.nodes[node_id]["level"]
    text_preview = ml.get_text(node_id)[:100]
    print(f"\n{node_id} (level {level}):")
    print(f"  {text_preview}...")

## Test 3: Drill down through hierarchy

In [ ]:
if top_nodes:
    root = top_nodes[0]
    print(f"Root: {root}")

    # Get children
    children = ml.get_children(root)
    print(f"\nRoot has {len(children)} children:")
    for child in children:
        print(f"  - {child} (level {ml.graph.nodes[child]['level']})")

    # Go one level deeper
    if children:
        first_child = children[0]
        grandchildren = ml.get_children(first_child)
        print(f"\nFirst child has {len(grandchildren)} children:")
        for gc in grandchildren:
            print(f"  - {gc}: {ml.get_text(gc)}")

## Test 4: Cascade test - add enough to trigger multiple levels

In [ ]:
# Fresh instance with more entries
ml2 = MemoryLayer(branching_factor=3)

# Add 27 entries (3^3 = should create 3 levels)
for i in range(27):
    ml2.add(f"Log entry {i}: some text content here")

stats = ml2.stats()
print(f"Total nodes: {stats['total_nodes']}")
print(f"Max level: {stats['max_level']}")
print(f"Nodes by level: {stats['nodes_by_level']}")
print(f"Buffer size: {stats['buffer_size']}")
print(f"Top nodes: {len(ml2.top())}")

## Test 5: Custom summary function

In [ ]:
def custom_summary(texts):
    """Just count how many items we're summarizing."""
    return f"[Summary of {len(texts)} items]"


ml3 = MemoryLayer(branching_factor=3, summary_fn=custom_summary)

for i in range(10):
    ml3.add(f"Item {i}")

# Check what the summaries look like
top = ml3.top()[0]
print(f"Top node text: {ml3.get_text(top)}")

children = ml3.get_children(top)
if children:
    print(f"First child text: {ml3.get_text(children[0])}")